In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from ../.env
load_dotenv(dotenv_path='../.env')

# Access the OPENAI_API_KEY
openai_api_key = os.getenv('OPENAI_API_KEY')
borsdata_api_key = os.getenv('BORSDATA_API_KEY')

if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY not found in the environment variables")
if borsdata_api_key is None:
    raise ValueError("BORSDATA_API_KEY not found in the environment variables")

In [13]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from playwright.sync_api import Page, expect
import re
from pydantic import BaseModel
from typing import Any

# Define a sample tool
def test_has_title(page: Page):
    page.goto("https://playwright.dev/")

    # Expect a title "to contain" a substring.
    expect(page).to_have_title(re.compile("Playwright"))

def test_get_started_link(page: Page):
    page.goto("https://playwright.dev/")

    # Click the get started link.
    page.get_by_role("link", name="Get started").click()

    # Expects page to have a heading with the name of Installation.
    expect(page.get_by_role("heading", name="Installation")).to_be_visible()

class PageInput(BaseModel):
    page: Any  # Using Any since we can't directly schema-tize the Page object
    
    class Config:
        arbitrary_types_allowed = True  # Allow arbitrary types like Page

# Modify the tool creation
title_tool = FunctionTool.from_defaults(
    fn=test_has_title,
    fn_schema=PageInput
)
link_tool = FunctionTool.from_defaults(
    fn=test_get_started_link,
    fn_schema=PageInput
)
# Initialize the LLM
llm = OpenAI(model="gpt-3.5-turbo")

# Create the ReAct agent with the tool and LLM
agent = ReActAgent.from_tools([title_tool, link_tool], llm=llm, verbose=True)

In [14]:
agent.chat("What is the title of the page?")

> Running step 0300b105-b1f7-4010-a6ac-c3dc45b414c8. Step input: What is the title of the page?
Thought: I can start by using the tool to check if the page has a title.
Action: test_has_title
Action Input: {'page': 'Page'}
Observation: Error: 'str' object has no attribute 'goto'
> Running step c7403add-fbed-4572-8aec-5cc54ea23d2a. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 3263e6e6-8711-428f-95d6-f85451a882ad. Step input: None
Thought: I can start by using the tool to check if the page has a title.
Action: test_has_title
Action Input: {}
Observation: Error: test_has_title() missing 1 required positional argument: 'page'
> Running step e140c4b6-addc-4fe8-abf0-cf911900097b. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The title of the page is not available at the moment.


AgentChatResponse(response='The title of the page is not available at the moment.', sources=[ToolOutput(content="Error: 'str' object has no attribute 'goto'", tool_name='test_has_title', raw_input={'kwargs': {'page': 'Page'}}, raw_output=AttributeError("'str' object has no attribute 'goto'"), is_error=True), ToolOutput(content='Error: Could not parse output. Please follow the thought-action-input format. Try again.', tool_name='unknown', raw_input={}, raw_output='Error: Could not parse output. Please follow the thought-action-input format. Try again.', is_error=False), ToolOutput(content="Error: test_has_title() missing 1 required positional argument: 'page'", tool_name='test_has_title', raw_input={'kwargs': {}}, raw_output=TypeError("test_has_title() missing 1 required positional argument: 'page'"), is_error=True)], source_nodes=[], is_dummy_stream=False, metadata=None)